In [2]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras import applications
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.python.keras
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import Flatten
from keras.preprocessing.image import img_to_array
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import TensorBoard
from time import time
from imutils import paths
import cv2
num_classes = 5
img_size = (224,224)
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
batch_size = 32
data = []
labels = []

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
        'ML/ML/DR 2.0/test',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical')

# Define the image directories for each class
image_dir = 'ML/ML/DR 2.0/train'

# Define the data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create separate data generators for each class, specifying augmentation only for folders 1-4
datagen_0 = ImageDataGenerator(rescale=1./255)  # No augmentation for class '0'

# Loop through each class and count images before augmentation
for class_label in os.listdir(image_dir):
    class_path = os.path.join(image_dir, class_label)

    # Count the number of images before augmentation
    before_augmentation_count = len(os.listdir(class_path))

    # Create data generators for each class
    if class_label == '0':
        data_generator = datagen_0
    else:
        data_generator = datagen
    
    train_generator = data_generator.flow_from_directory(
        class_path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        classes=[class_label],#[class_label]
        shuffle=False  # Disable shuffling for accurate counting
    )

    
# train_generator_0 = datagen_0.flow_from_directory(
#     image_dir,
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical',  
#     classes=['0']
# )

# train_generator_1_4 = datagen.flow_from_directory(
#     image_dir,
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical',  
#     classes=['1', '2', '3', '4']
# )
import random
import cv2
data = []
labels = []

imagePaths = sorted(list(paths.list_images( "C:\\Users\\Nishi Mahajan\\Desktop\\python\\ML\\ML\\DR\\" )))
                                           
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    if image is None:
        continue

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    clahe_image = clahe.apply(image)
    # Apply Canny edge detection
    edges = cv2.Canny(clahe_image, 30, 150)  # Adjust thresholds as needed

    # Apply thresholding to segment retinal features
    _, binary_image = cv2.threshold(edges, 50, 255, cv2.THRESH_BINARY)

    # Apply morphological opening for noise reduction
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)

    # Apply vessel segmentation using morphological operations
    kernel = np.ones((5, 5), np.uint8)
    vessel_segmentation = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    vessel_segmentation = cv2.morphologyEx(
    vessel_segmentation, cv2.MORPH_OPEN, kernel)

    # # Save the preprocessed image and vessel segmentation result
    # cv2.imwrite('preprocessed_retinal_image.jpg', vessel_segmentation)
    new_image = img_to_array(vessel_segmentation)
    data.append(new_image)

    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

# loop over the input images
# for imagePath in imagePaths:
#     # load the image, pre-process it, and store it in the data list
#     image = cv2.imread(imagePath)
#     if image is None:
#         continue
#     image = cv2.resize(image, img_size)
#     image = img_to_array(image)
#     data.append(image)
 
#  # extract the class label from the image path and update the labels list
#     label = imagePath.split(os.path.sep)[-2]
# # label = 0 if label == "X" else 1
#     labels.append(label)
print(labels)
imagePaths
data[10]
plt.imshow(data[30]/255)
data[0].shape
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))
model.summary()
for layers in model.layers:
    layers.trainable = False
model.summary()
l = model.output

l = Flatten()(l)
# l = GlobalAveragePooling2D()(l)
l = Dense(128, activation="relu")(l) # Adding a single dense layer over the pre-built model

predictions = Dense(num_classes, activation="softmax")(l)
final_model = Model(inputs=model.input, outputs=predictions)
final_model.summary()
final_model.compile(optimizer='adam', metrics=["accuracy"], loss="categorical_crossentropy")
NAME = "VGG16"
tensorboard = TensorBoard(log_dir='logs_vgg/{}'.format(NAME))
filepath = "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [tensorboard,es, checkpoint]
import math
total_samples = len(train_generator.filenames)
steps_per_epoch = math.ceil(total_samples / batch_size)

validation_data_length = len(validation_generator.filenames)
validation_steps = validation_data_length // batch_size
print(steps_per_epoch) 
print(validation_steps)
final_model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch, 
        epochs=10,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=callbacks_list)

final_model.save('complete_model.h5')
from keras.models import load_model 


loadedmodel=load_model("complete_model.h5")
#load_model.load_weights("first_try.h5")
print("Loaded model from disk")
loadedmodel.summary()
import numpy as np
from keras.preprocessing import image

# img_pred = image.load_img('ML/ML/DR/single_prediction/Proliferative_DR_EDA01.jpeg', target_size = img_size)
# img_pred = img_pred.reshape(224, 224, 3)
# img_pred = np.mean(img_pred, axis=0)
image = cv2.imread('ML/ML/DR/single_prediction/Moderate_DR_222.png')
img_pred = np.array(image)
plt.imshow(img_pred)
# img_pred = image.img_to_array(img_pred)
# img_pred = np.squeeze(img_pred, axis=0)
img_pred = cv2.resize(img_pred, (224, 224))
img_pred = np.expand_dims(img_pred, axis = 0)
img_pred = img_pred / 255.0 
rslt = loadedmodel.predict(img_pred)

# ind = train_generator.class_indices
class_indices = train_generator.class_indices
# predicted_class = labels[np.argmax(predictions)]
predicted_class_label = list(class_indices.keys())[np.argmax(rslt)]

print("Predicted class:", predicted_class_label)